# Training a model for catchment classification.
* kategoryzacja zlewni oparta o wielkość zlewni i wyniki oblczeń zlewni
* opad nie jest przyjmowany ponieważ jest tylko jedna wartość i jego wartość jest standaryzowana do zera

# Imports

In [2]:
import swmmio
import pyswmm
import pandas as pd
import numpy as np
import plotly.graph_objects as go

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder, MinMaxScaler
from sklearn.compose import ColumnTransformer
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping

from plotly.subplots import make_subplots


desired_width = 500
pd.set_option("display.width", desired_width)
np.set_printoptions(linewidth=desired_width)
pd.set_option("display.max_columns", 30)

## Catchment categories

In [3]:
classes = pd.DataFrame(
    data={
        "classes": [
            "compact_urban_development",
            "urban",
            "loose_urban_development",
            "wooded_area",
            "grassy",
            "loose_soil",
            "steep_area",
        ]
    }
)
classes

,classes
0,compact_urban_development
1,urban
2,loose_urban_development
3,wooded_area
4,grassy
5,loose_soil
6,steep_area


## Get files

In [4]:
INP_FILE = "dataset/subcatchment_dataset.inp"
RPT_FILE = "dataset/subcatchment_dataset.inp"

## Run simulation

In [6]:
with pyswmm.Simulation(INP_FILE) as sim:
    for step in sim:
        pass

Exception: 
  ERROR 303: cannot open input file.

## Read inp and rpt file as swmmio model object

In [ ]:
model = swmmio.Model(INP_FILE)

## Get subcatchments data from the model

In [ ]:
raw_subcatchments = model.subcatchments.dataframe
subcatchments = raw_subcatchments.copy()
subcatchments[:5]

,Raingage,Outlet,Area,PercImperv,Width,PercSlope,CurbLength,N-Imperv,N-Perv,S-Imperv,S-Perv,PctZero,RouteTo,TotalPrecip,TotalRunon,TotalEvap,TotalInfil,ImpervRunoff,PervRunoff,TotalRunoffIn,TotalRunoffMG,PeakRunoff,RunoffCoeff,coords
Name,,,,,,,,,,,,,,,,,,,,,,,,
S1,Raingage2,O1,0.50,20.00,300.00,5.00,0,0.150,0.41,1.27,5.08,70,OUTLET,10.10,0.00,0.00,5.91,1.95,2.17,4.12,0.02,0.02,0.407,"[(777180.0, 592590.0), (777180.0, 592585.0), (777175.0, 592585.0), (777175.0, 592590.0), (777180.0, 592590.0)]"
S10,Raingage2,O1,1.87,45.00,136.75,15.00,0,0.013,0.15,1.27,5.08,90,OUTLET,10.10,0.00,0.00,4.13,4.50,1.43,5.93,0.11,0.09,0.587,NaN
S100,Raingage2,O1,1.71,45.00,130.77,10.00,0,0.013,0.15,1.27,5.08,90,OUTLET,10.10,0.00,0.00,4.18,4.50,1.38,5.87,0.10,0.08,0.581,NaN
S1000,Raingage2,O1,0.68,10.00,82.46,5.09,0,0.150,0.41,1.27,5.08,10,OUTLET,10.10,0.00,0.00,7.73,0.90,1.36,2.26,0.02,0.01,0.224,NaN
S1001,Raingage2,O1,0.26,83.33,50.99,45.00,0,0.013,0.05,1.27,5.08,80,OUTLET,10.10,0.00,0.00,1.04,8.23,0.65,8.88,0.02,0.02,0.879,NaN


## Drop unused columns

In [ ]:
subcatchments.columns

Index(['Raingage', 'Outlet', 'Area', 'PercImperv', 'Width', 'PercSlope', 'CurbLength', 'N-Imperv', 'N-Perv', 'S-Imperv', 'S-Perv', 'PctZero', 'RouteTo', 'TotalPrecip', 'TotalRunon', 'TotalEvap', 'TotalInfil', 'ImpervRunoff', 'PervRunoff', 'TotalRunoffIn', 'TotalRunoffMG', 'PeakRunoff', 'RunoffCoeff', 'coords'], dtype='object')

In [ ]:
subcatchments.drop(['Raingage',  'Outlet', 'PercImperv', 'Width', 'PercSlope', 'CurbLength', 'N-Imperv', 'N-Perv', 'S-Imperv', 'S-Perv', 'PctZero', 'RouteTo', 'coords'], axis=1, inplace=True)
subcatchments

,Area,TotalPrecip,TotalRunon,TotalEvap,TotalInfil,ImpervRunoff,PervRunoff,TotalRunoffIn,TotalRunoffMG,PeakRunoff,RunoffCoeff
Name,,,,,,,,,,,
S1,0.50,10.10,0.00,0.00,5.91,1.95,2.17,4.12,0.02,0.02,0.407
S10,1.87,10.10,0.00,0.00,4.13,4.50,1.43,5.93,0.11,0.09,0.587
S100,1.71,10.10,0.00,0.00,4.18,4.50,1.38,5.87,0.10,0.08,0.581
S1000,0.68,10.10,0.00,0.00,7.73,0.90,1.36,2.26,0.02,0.01,0.224
S1001,0.26,10.10,0.00,0.00,1.04,8.23,0.65,8.88,0.02,0.02,0.879
...,...,...,...,...,...,...,...,...,...,...,...
S995,1.10,10.10,0.00,0.00,7.53,0.90,1.56,2.46,0.03,0.02,0.243
S996,1.89,10.10,0.00,0.00,7.55,0.46,2.04,2.49,0.05,0.03,0.247
S997,1.19,10.10,0.00,0.00,7.91,0.90,1.18,2.08,0.02,0.01,0.206


## Get categories

In [ ]:
# print([attr for attr in dir(model.inp) if not attr.startswith("_")])
categories = model.inp.tags
categories

,Name,Tag
ElementType,,
Subcatch,S1,#comment
Subcatch,S2,compact_urban_development
Subcatch,S3,loose_soil
Subcatch,S4,wooded_area
Subcatch,S5,loose_urban_development
...,...,...
Subcatch,S997,grassy
Subcatch,S998,steep_area
Subcatch,S999,urban


### Add categories column to subcatchments DataFrame

In [ ]:
# Reset the index of subcatchments DataFrame
subcatchments.reset_index(inplace=True)

# Merge the two DataFrames on the "Name" column
merged_df = subcatchments.merge(model.inp.tags, left_on="Name", right_on="Name", how="left")

# Rename the 'Tag' column to 'categories'
merged_df.rename(columns={"Tag": "categories"}, inplace=True)

# Set the index back to "Name"
merged_df.set_index("Name", inplace=True)

In [ ]:
merged_df[:3]

,Area,TotalPrecip,TotalRunon,TotalEvap,TotalInfil,ImpervRunoff,PervRunoff,TotalRunoffIn,TotalRunoffMG,PeakRunoff,RunoffCoeff,categories
Name,,,,,,,,,,,,
S1,0.50,10.10,0.00,0.00,5.91,1.95,2.17,4.12,0.02,0.02,0.407,#comment
S10,1.87,10.10,0.00,0.00,4.13,4.50,1.43,5.93,0.11,0.09,0.587,compact_urban_development
S100,1.71,10.10,0.00,0.00,4.18,4.50,1.38,5.87,0.10,0.08,0.581,compact_urban_development


# Split data into features and target

In [ ]:
X = merged_df.drop("categories", axis=1)
y = merged_df["categories"]

print(X.shape, y.shape)

(1001, 11) (1001,)


In [ ]:
X.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1001 entries, S1 to S999
Data columns (total 11 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Area           1001 non-null   float64
 1   TotalPrecip    1001 non-null   object 
 2   TotalRunon     1001 non-null   object 
 3   TotalEvap      1001 non-null   object 
 4   TotalInfil     1001 non-null   object 
 5   ImpervRunoff   1001 non-null   float64
 6   PervRunoff     1001 non-null   object 
 7   TotalRunoffIn  1001 non-null   float64
 8   TotalRunoffMG  1001 non-null   float64
 9   PeakRunoff     1001 non-null   float64
 10  RunoffCoeff    1001 non-null   float64
dtypes: float64(6), object(5)
memory usage: 93.8+ KB


In [ ]:
X["TotalPrecip"] = pd.to_numeric(X["TotalPrecip"])
X["TotalRunon"] = pd.to_numeric(X["TotalRunon"])
X["TotalEvap"] = pd.to_numeric(X["TotalEvap"])
X["TotalInfil"] = pd.to_numeric(X["TotalInfil"])
X["PervRunoff"] = pd.to_numeric(X["PervRunoff"])

In [ ]:
X.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1001 entries, S1 to S999
Data columns (total 11 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Area           1001 non-null   float64
 1   TotalPrecip    1001 non-null   float64
 2   TotalRunon     1001 non-null   float64
 3   TotalEvap      1001 non-null   float64
 4   TotalInfil     1001 non-null   float64
 5   ImpervRunoff   1001 non-null   float64
 6   PervRunoff     1001 non-null   float64
 7   TotalRunoffIn  1001 non-null   float64
 8   TotalRunoffMG  1001 non-null   float64
 9   PeakRunoff     1001 non-null   float64
 10  RunoffCoeff    1001 non-null   float64
dtypes: float64(11)
memory usage: 93.8+ KB


In [ ]:
X = pd.get_dummies(X, drop_first=True)

In [ ]:
X.head()

,Area,TotalPrecip,TotalRunon,TotalEvap,TotalInfil,ImpervRunoff,PervRunoff,TotalRunoffIn,TotalRunoffMG,PeakRunoff,RunoffCoeff
Name,,,,,,,,,,,
S1,0.50,10.1,0.0,0.0,5.91,1.95,2.17,4.12,0.02,0.02,0.407
S10,1.87,10.1,0.0,0.0,4.13,4.50,1.43,5.93,0.11,0.09,0.587
S100,1.71,10.1,0.0,0.0,4.18,4.50,1.38,5.87,0.10,0.08,0.581
S1000,0.68,10.1,0.0,0.0,7.73,0.90,1.36,2.26,0.02,0.01,0.224
S1001,0.26,10.1,0.0,0.0,1.04,8.23,0.65,8.88,0.02,0.02,0.879


## Data standarization

In [ ]:
scaler = MinMaxScaler()
X_normalized = scaler.fit_transform(X)
X_normalized_df = pd.DataFrame(X_normalized, columns=X.columns, index=X.index)

In [ ]:
X_normalized_df.head()

,Area,TotalPrecip,TotalRunon,TotalEvap,TotalInfil,ImpervRunoff,PervRunoff,TotalRunoffIn,TotalRunoffMG,PeakRunoff,RunoffCoeff
Name,,,,,,,,,,,
S1,0.246231,0.0,0.0,0.0,0.608479,0.216687,0.683871,0.392630,0.111111,0.142857,0.391528
S10,0.934673,0.0,0.0,0.0,0.386534,0.534247,0.445161,0.622618,0.611111,0.642857,0.622593
S100,0.854271,0.0,0.0,0.0,0.392768,0.534247,0.429032,0.614994,0.555556,0.571429,0.614891
S1000,0.336683,0.0,0.0,0.0,0.835411,0.085928,0.422581,0.156290,0.111111,0.071429,0.156611
S1001,0.125628,0.0,0.0,0.0,0.001247,0.998755,0.193548,0.997459,0.111111,0.142857,0.997433


## Encode target labels

In [ ]:
encoder = OneHotEncoder(sparse_output=False)
y_encoded = encoder.fit_transform(y.values.reshape(-1, 1))

In [ ]:
y_encoded

array([[1., 0., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 1., ..., 0., 0., 0.],
       [0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 0., 1., 0.]])

## Split the data into training and testing sets

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_normalized_df, y_encoded, test_size=0.2, random_state=42)

print("X_train shape:", X_train.shape)
print("y_train shape:", y_train.shape)
print("X_test shape:", X_test.shape)
print("y_test shape:", y_test.shape)

X_train shape: (800, 11)
y_train shape: (800, 8)
X_test shape: (201, 11)
y_test shape: (201, 8)


# Build the model

In [ ]:
model = Sequential()
model.add(Dense(X_train.shape[1], activation='relu', input_dim=X_train.shape[1]))
model.add(Dropout(0.2))
model.add(Dense(16, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(units=y_train.shape[1], activation='softmax'))
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_3 (Dense)             (None, 11)                132       
                                                                 
 dropout_2 (Dropout)         (None, 11)                0         
                                                                 
 dense_4 (Dense)             (None, 16)                192       
                                                                 
 dropout_3 (Dropout)         (None, 16)                0         
                                                                 
 dense_5 (Dense)             (None, 8)                 136       
                                                                 
Total params: 460
Trainable params: 460
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

In [ ]:
history = model.fit(
    x=X_train,
    y=y_train,
    epochs=250,
    validation_data=(X_test, y_test),
    validation_split=0.2,
    verbose=1,
    batch_size=16,
    callbacks=[EarlyStopping(monitor='val_loss', patience=10)]
)

Epoch 1/250
50/50 [==============================] - 1s 7ms/step - loss: 2.0196 - accuracy: 0.2212 - val_loss: 1.9392 - val_accuracy: 0.3781
Epoch 2/250
50/50 [==============================] - 0s 3ms/step - loss: 1.8509 - accuracy: 0.3587 - val_loss: 1.7366 - val_accuracy: 0.3781
Epoch 3/250
50/50 [==============================] - 0s 3ms/step - loss: 1.6780 - accuracy: 0.3775 - val_loss: 1.5824 - val_accuracy: 0.3781
Epoch 4/250
50/50 [==============================] - 0s 3ms/step - loss: 1.5756 - accuracy: 0.4137 - val_loss: 1.4871 - val_accuracy: 0.4328
Epoch 5/250
50/50 [==============================] - 0s 3ms/step - loss: 1.4631 - accuracy: 0.4600 - val_loss: 1.3961 - val_accuracy: 0.4378
Epoch 6/250
50/50 [==============================] - 0s 3ms/step - loss: 1.3733 - accuracy: 0.4638 - val_loss: 1.3118 - val_accuracy: 0.4378
Epoch 7/250
50/50 [==============================] - 0s 3ms/step - loss: 1.2914 - accuracy: 0.4900 - val_loss: 1.2371 - val_accuracy: 0.4478
Epoch 8/250
5

In [ ]:
# Evaluate ANN model
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Test set accuracy: {accuracy:.2f}")

7/7 [==============================] - 0s 2ms/step - loss: 0.0342 - accuracy: 0.9950
Test set accuracy: 1.00


In [ ]:
y_pred = model.predict(X_test)

7/7 [==============================] - 0s 1ms/step


In [ ]:
metrics = pd.DataFrame(history.history)
metrics['epoch'] = history.epoch
metrics

,loss,accuracy,val_loss,val_accuracy,epoch
0,2.019596,0.22125,1.939187,0.378109,0
1,1.850854,0.35875,1.736618,0.378109,1
2,1.678047,0.37750,1.582397,0.378109,2
3,1.575553,0.41375,1.487055,0.432836,3
4,1.463140,0.46000,1.396128,0.437811,4
...,...,...,...,...,...
191,0.191253,0.94500,0.037950,0.995025,191
192,0.225967,0.92500,0.040917,0.990050,192
193,0.174037,0.95250,0.038070,0.995025,193
194,0.205539,0.93750,0.036258,0.995025,194


# Model evaluate

In [ ]:
fig = make_subplots(rows=1, cols=2)
fig.add_trace(go.Scatter(x=metrics['epoch'], y=metrics['accuracy'], name='accuracy'), row=1, col=1)
fig.add_trace(go.Scatter(x=metrics['epoch'], y=metrics['loss'], name='loss'), row=1, col=2)
fig.add_trace(go.Scatter(x=metrics['epoch'], y=metrics['val_accuracy'], name='val_accuracy'), row=1, col=1)
fig.add_trace(go.Scatter(x=metrics['epoch'], y=metrics['val_loss'], name='val_loss'), row=1, col=2)

fig.update_xaxes(title_text='epochs')
# fig.update_yaxes(title_text='accuracy')
fig.update_layout(width=1000, title='Accuracy and Loss')
fig.show()

In [ ]:
test_loss, test_acc = model.evaluate(X_test, y_test, verbose=0)
print(test_acc)

0.9950248599052429


Predykcja na podstawie modelu.
* model.evaluate(y_true, y_pred) - pozwala obliczyć metryki modelu
* model.predict_classes() - pozwala zwrócić odpowiednio przewidziane klasy
* model.predict_proba(), model.predict() - pozwala zwrócić prawdopodobieństwo danej klasy

In [ ]:
predictions = model.predict(X_test)
predictions

7/7 [==============================] - 0s 2ms/step


array([[4.92109962e-07, 6.88161472e-11, 9.95627522e-01, ..., 1.81500446e-18, 1.64066535e-06, 3.38391797e-03],
       [4.20155819e-04, 1.93274973e-05, 6.12591021e-03, ..., 1.78349072e-11, 1.51135772e-02, 1.41123533e-02],
       [1.39038161e-06, 5.24899360e-11, 9.94987726e-01, ..., 2.14634261e-17, 2.03694162e-06, 5.19080961e-04],
       ...,
       [9.82645632e-10, 6.92632085e-08, 1.95395318e-03, ..., 3.57672130e-14, 1.85587183e-07, 9.98019814e-01],
       [3.09350253e-06, 1.13632007e-10, 9.90246177e-01, ..., 2.83596636e-16, 3.12111183e-06, 3.17870581e-04],
       [3.34792491e-03, 1.25695615e-05, 1.16208650e-03, ..., 2.53215626e-11, 6.45784214e-02, 5.28679986e-04]], dtype=float32)

In [ ]:
predictions_df = pd.DataFrame(predictions)
predictions_df

,0,1,2,3,4,5,6,7
0,4.921100e-07,6.881615e-11,0.995628,7.806099e-04,0.000206,1.815004e-18,1.640665e-06,0.003384
1,4.201558e-04,1.932750e-05,0.006126,1.377076e-03,0.962831,1.783491e-11,1.511358e-02,0.014112
2,1.390382e-06,5.248994e-11,0.994988,4.336103e-03,0.000154,2.146343e-17,2.036942e-06,0.000519
3,5.782059e-07,1.467719e-11,0.996893,2.409827e-03,0.000113,9.516357e-19,1.136163e-06,0.000582
4,3.746226e-06,5.332866e-10,0.993523,5.089581e-03,0.000300,1.319095e-15,4.983337e-06,0.001078
...,...,...,...,...,...,...,...,...
196,1.843317e-09,3.380320e-08,0.003380,2.433424e-09,0.000052,2.498490e-15,2.061183e-07,0.996568
197,4.664551e-06,1.247979e-10,0.983400,1.624174e-02,0.000168,8.768756e-16,3.576408e-06,0.000182
198,9.826456e-10,6.926321e-08,0.001954,6.338413e-10,0.000026,3.576721e-14,1.855872e-07,0.998020
199,3.093503e-06,1.136320e-10,0.990246,9.254303e-03,0.000176,2.835966e-16,3.121112e-06,0.000318


In [ ]:
predictions_cls = predictions.argmax(axis=-1)
predictions_cls

array([2, 4, 2, 2, 2, 1, 2, 7, 3, 2, 3, 7, 2, 5, 5, 1, 5, 4, 2, 3, 7, 3, 7, 4, 7, 1, 2, 4, 6, 1, 2, 6, 6, 4, 6, 1, 7, 2, 7, 4, 2, 5, 7, 2, 2, 2, 7, 2, 3, 4, 1, 1, 4, 1, 3, 3, 7, 2, 7, 3, 2, 7, 2, 6, 7, 2, 3, 1, 3, 2, 2, 2, 4, 2, 2, 2, 1, 2, 2, 1, 7, 2, 2, 2, 6, 2, 5, 7, 2, 7, 6, 2, 3, 4, 2, 2, 2, 2, 6, 6, 3, 2, 6, 4, 5, 3, 6, 3, 2, 2, 1, 2, 2, 2, 6, 7, 3, 1, 2, 2, 6, 7, 2, 2, 6, 3, 2, 2, 3, 5, 4, 1, 6, 2, 1, 4, 7, 3, 3, 7, 6, 2, 7, 2, 2, 2, 4, 2, 4, 2, 3, 5, 2, 6, 1, 2, 5, 2, 4, 2, 4, 6, 2, 3,
       6, 5, 2, 1, 4, 2, 6, 2, 1, 5, 1, 2, 4, 2, 3, 2, 7, 1, 2, 3, 5, 6, 6, 1, 4, 2, 2, 2, 7, 7, 2, 1, 7, 2, 7, 2, 4], dtype=int64)

# Zapisanie modelu

In [ ]:
model.save(r'C:\Users\Dell\Documents\Git\stormwater-analysis\stormwater_analysis\data\area_rpt')

INFO:tensorflow:Assets written to: C:\Users\Dell\Documents\Git\stormwater-analysis\stormwater_analysis\data\area_rpt\assets


# Załadowanie modelu

In [ ]:
from tensorflow import keras
model = keras.models.load_model(r'C:\Users\Dell\Documents\Git\stormwater-analysis\stormwater_analysis\data\area_rpt')

In [ ]:
pred = model.predict(X_test)
pred

7/7 [==============================] - 0s 2ms/step


array([[4.92109962e-07, 6.88161472e-11, 9.95627522e-01, ..., 1.81500446e-18, 1.64066535e-06, 3.38391797e-03],
       [4.20155819e-04, 1.93274973e-05, 6.12591021e-03, ..., 1.78349072e-11, 1.51135772e-02, 1.41123533e-02],
       [1.39038161e-06, 5.24899360e-11, 9.94987726e-01, ..., 2.14634261e-17, 2.03694162e-06, 5.19080961e-04],
       ...,
       [9.82645632e-10, 6.92632085e-08, 1.95395318e-03, ..., 3.57672130e-14, 1.85587183e-07, 9.98019814e-01],
       [3.09350253e-06, 1.13632007e-10, 9.90246177e-01, ..., 2.83596636e-16, 3.12111183e-06, 3.17870581e-04],
       [3.34792491e-03, 1.25695615e-05, 1.16208650e-03, ..., 2.53215626e-11, 6.45784214e-02, 5.28679986e-04]], dtype=float32)